Loading NAICS data

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data_dir = './A_NEWLY_HIRED_DATA_ANALYST/'

# Loading LMO_Detailed_Industries_by_NAICS data
lmo_detailed_industries_data = pd.read_excel(data_dir+'LMO_Detailed_Industries_by_NAICS.xlsx')
lmo_detailed_industries_data.head()

In [ ]:
# create a list of NAICS for industries
naic_lists = lmo_detailed_industries_data['NAICS'].astype(str).str.replace(' &', ',').str.split(', ')
lmo_detailed_industries_data['NAICS_list'] = naic_lists
lmo_detailed_industries_data.head()

2 Digit NAICS Industries

In [ ]:
# Get the data of 2digit NAICS industries
dataframe_2_naics = pd.read_csv(data_dir+'RTRA_Employ_2NAICS_00_05.csv')


file_2_naics = ['RTRA_Employ_2NAICS_06_10.csv', 'RTRA_Employ_2NAICS_11_15.csv',
                'RTRA_Employ_2NAICS_16_20.csv', 'RTRA_Employ_2NAICS_97_99.csv']


for file_p in file_2_naics:
    df = pd.read_csv(data_dir+file_p)
    dataframe_2_naics = dataframe_2_naics.append(df, ignore_index=True)

dataframe_2_naics.shape

In [ ]:
dataframe_2_naics.head()

In [ ]:
# Separate the Industry description and NAICS code then get the lower code and upper code
df1 = pd.DataFrame(dataframe_2_naics['NAICS'].str.split('[').tolist(), columns=["NAICS", "CODE"])
df1 = pd.DataFrame(df1['CODE'].str.replace(']', '').str.split('-').tolist(), columns=["lower_code", "upper_code"])
df1.head()

In [ ]:
# Append the lower_code and upper_code columns to the 2digits NAICS dataframe
dataframe_2_naics["lower_code"] = df1["lower_code"]
dataframe_2_naics["upper_code"] = df1["upper_code"]
dataframe_2_naics.head()

In [ ]:
# Function to get the 'LMO_Detailed_Industry' values for a NAICS code in RTRA files
def add_lmo_industry(df):
    lmo_df = lmo_detailed_industries_data.apply(lambda y: y["LMO_Detailed_Industry"] 
                                                if ((df['lower_code'] in y['NAICS_list']) or (df['upper_code'] in y['NAICS_list']))
                                                else np.nan, axis=1)
    lmo_df = lmo_df.dropna(how='all', axis=0)
    if lmo_df.empty:
        lmo_df = np.nan
    else:
        lmo_df = lmo_df.to_string(index=False)
    return lmo_df

In [ ]:
# Get the LMO_Detailed_Industry for the 2digit NAICS RTRA file
dataframe_2_naics["LMO_Detailed_Industry"] = dataframe_2_naics.apply(add_lmo_industry, axis=1)
dataframe_2_naics.head(10)

3 Digit NAICS Industry

In [ ]:
# Get the data of 3-digit NAICS industries
dataframe_3_naics = pd.read_csv(data_dir+'RTRA_Employ_3NAICS_00_05.csv')


file_3_naics = ['RTRA_Employ_3NAICS_06_10.csv', 'RTRA_Employ_3NAICS_11_15.csv',
                'RTRA_Employ_3NAICS_16_20.csv', 'RTRA_Employ_3NAICS_97_99.csv']


for file_p in file_3_naics:
    df = pd.read_csv(data_dir+file_p)
    dataframe_3_naics = dataframe_3_naics.append(df, ignore_index=True)

dataframe_3_naics.shape

In [ ]:
# Separate the Industry description and NAICS code
df1 = pd.DataFrame(dataframe_3_naics['NAICS'].str.split('[').tolist(), columns=["NAICS", "CODE"])
df1["CODE"] = df1['CODE'].str.replace(']', '')
df1.head()

In [ ]:
# Maintaining the table consistent as of 2-digit NAICS dataframe
dataframe_3_naics["lower_code"] = df1["CODE"]
dataframe_3_naics["upper_code"] = None
dataframe_3_naics.head()

In [ ]:
# Get the LMO_Detailed_Industry for the 3-digit NAICS RTRA file
dataframe_3_naics["LMO_Detailed_Industry"] = dataframe_3_naics.apply(add_lmo_industry, axis=1)
dataframe_3_naics.head(10)

4 Digit NAICS Industries

In [ ]:
# Get the data of 4-digit NAICS industries
dataframe_4_naics = pd.read_csv(data_dir+'RTRA_Employ_4NAICS_00_05.csv')


file_4_naics = ['RTRA_Employ_4NAICS_06_10.csv', 'RTRA_Employ_4NAICS_11_15.csv',
                'RTRA_Employ_4NAICS_16_20.csv', 'RTRA_Employ_4NAICS_97_99.csv']


for file_p in file_4_naics:
    df = pd.read_csv(data_dir+file_p)
    dataframe_4_naics = dataframe_4_naics.append(df, ignore_index=True)

print(dataframe_4_naics.shape)
print(dataframe_4_naics.head())

In [ ]:
# Maintaning the shape of dataframe_4_naics as of dataframe_2_naics
dataframe_4_naics["lower_code"] = dataframe_4_naics["NAICS"]
dataframe_4_naics["upper_code"] = None
dataframe_4_naics.head()

In [ ]:
# Get the LMO_Detailed_Industry for the 4-digits NAICS RTRA file
dataframe_4_naics["LMO_Detailed_Industry"] = dataframe_4_naics.apply(add_lmo_industry, axis=1)
dataframe_4_naics.head(10)

In [ ]:
cols = ["SYEAR", "SMTH", "LMO_Detailed_Industry", "_EMPLOYMENT_"]

# Creating a single dataframe with the columns Year, Month and LMO Industry and Employment from all the 2, 3 and 4 digits NAICS
naics_employment_detail_df = dataframe_2_naics[cols]
naics_employment_detail_df = naics_employment_detail_df.append(dataframe_3_naics[cols], ignore_index=True)
naics_employment_detail_df = naics_employment_detail_df.append(dataframe_4_naics[cols], ignore_index=True)

print(naics_employment_detail_df.shape)
print(naics_employment_detail_df.head(10))

In [ ]:
# drop rows with NaN values
naics_employment_detail_df = naics_employment_detail_df.dropna(axis=0, how='any')

print(naics_employment_detail_df.shape)
print(naics_employment_detail_df.head(10))

In [ ]:
# Calculate the Employment summary by Year, Month and LOM Industry
naics_employment_summary = naics_employment_detail_df.groupby(["SYEAR", "SMTH", "LMO_Detailed_Industry"], as_index=False).sum()
print(naics_employment_summary.shape)
naics_employment_summary.head()

In [ ]:
naics_employment_summary.tail(10)

In [ ]:
# Read 'Data_Output_Template' file
data_output = pd.read_excel(data_dir+'Data_Output_Template.xlsx')
print(data_output.shape)
print(data_output.head())

In [ ]:
# Crate Year, Month and LMO_Detailed_industry combined idx to get the data_output formated result
naics_employment_summary1 = naics_employment_summary.copy()
naics_employment_summary1['idx'] = naics_employment_summary1["SYEAR"].astype(str) + '-' + naics_employment_summary1["SMTH"].astype(str) + '-' + naics_employment_summary1["LMO_Detailed_Industry"]
print(naics_employment_summary1.head())

data_output1 = data_output.copy()
data_output1['idx'] = data_output1["SYEAR"].astype(str) + '-' + data_output1["SMTH"].astype(str) + '-' + data_output1["LMO_Detailed_Industry"]
print(data_output1.head())

In [ ]:
# Merge the two dataframes data_output1 and naics_employment_summary1
combined_data = pd.merge(data_output1, naics_employment_summary1, left_on='idx', right_on='idx', how='left')
print(combined_data.shape)
print(combined_data.head())

In [ ]:
# Fille tha NaN values with zero in '_EMPLOYMENT_' column
combined_data["_EMPLOYMENT_"] = combined_data["_EMPLOYMENT_"].fillna(0)
combined_data.head()

In [ ]:
# Get the month wise employment summary data into "Employment" column of dat_output dataframe
data_output["Employment"] = combined_data["_EMPLOYMENT_"].astype(np.int)
print(data_output.info())
data_output.head()

In [ ]:
# Export data_output to excel file
data_output.to_excel('Data_Output.xlsx', header=True, index=False)

Exploratory Data Analysis

In [ ]:
# create a dataframe with industry wise employment summary
industry_wise_summary = data_output.groupby(["LMO_Detailed_Industry"])["Employment"].sum()
industry_wise_summary.head()

In [ ]:
industry_wise_summary.plot(kind="barh", figsize=(20,25))

In [ ]:
# Plotting employment wise top 10 Industries.

industry_wise_summary.sort_values(ascending=False)[:10].plot(kind='barh')
plt.xlabel("Employment")
plt.title("Employment wise Top 10 Industries Bar plot")

In [ ]:
# Create a dataframe with Year and Month as index
month_wise_employment_summary = data_output.copy()
month_wise_employment_summary['month_idx'] = pd.to_datetime([f'{y}-{m}' for y, m in zip(month_wise_employment_summary.SYEAR, month_wise_employment_summary.SMTH)])
month_wise_employment_summary.index = month_wise_employment_summary["month_idx"]
month_wise_employment_summary.head()

In [ ]:
construction_data = month_wise_employment_summary[month_wise_employment_summary["LMO_Detailed_Industry"] == "Construction"]
construction_data.head()

In [ ]:
construction_data.plot(y="Employment", title="Employment in Constction evolved overtime", figsize=(20,10))
plt.xlabel("Month and Year")
plt.ylabel("Employment")

In [ ]:
total_employment_summary = month_wise_employment_summary.groupby("month_idx")["Employment"].sum()
total_employment_summary = total_employment_summary.reset_index()
# total_employment_summary.head()
plt.figure(figsize=(20,10))
sns.lineplot(x="month_idx", y="Employment", data=total_employment_summary, label="Total Employment")
sns.lineplot(x="month_idx", y="Employment", data=construction_data, label="Construction Employment")
plt.title("")
plt.show()

In [ ]:
# Calculating the percentage of Employment contributed by Construction Industry
construction_perc_df = pd.merge(left=total_employment_summary, right=construction_data, left_on="month_idx", right_on="month_idx", how="left")
construction_perc_df["Employment_perc"] = construction_perc_df["Employment_y"] / construction_perc_df["Employment_x"] * 100
construction_perc_df.head()

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x="month_idx", y="Employment_perc", data=construction_perc_df)
plt.xlabel("Year")
plt.ylabel("Employment Percentage")
plt.title("Month wise Employment Percentage Contribution by Construction Industry")
plt.show()

In [ ]:
lmo_detailed_industries_data[lmo_detailed_industries_data["LMO_Detailed_Industry"] == "Construction"]

In [ ]:
# Subsectors contibution towards the Construction Industry Sector
construction_subsector_data = dataframe_3_naics[dataframe_3_naics["lower_code"].str.match(r'23[0-9]') == True]
construction_subsector_summary = construction_subsector_data.groupby(["SYEAR", "NAICS"])["_EMPLOYMENT_"].sum()
construction_subsector_summary = construction_subsector_summary.reset_index()
construction_subsector_summary.head()

In [ ]:
plt.figure(figsize=(50,20))
sns.barplot(x="SYEAR", y="_EMPLOYMENT_", hue="NAICS", data=construction_subsector_summary)
plt.xlabel("Year")
plt.ylabel("Employment")
plt.title("Year wise employment contribution by Subsector of Construction Sector")
plt.show()

In [ ]:
# Subsectors contibution towards the Construction Industry Sector
construction_subsector = construction_subsector_data.groupby(["NAICS"])["_EMPLOYMENT_"].sum()
construction_subsector = construction_subsector.reset_index()
construction_subsector.head()

In [ ]:
plt.figure(figsize=(15,5))
# construction_subsector.plot(kind="bar")
sns.barplot(x="NAICS", y="_EMPLOYMENT_", data=construction_subsector)
plt.ylabel("Employment")
plt.title("Employment contribution by Subsector of Construction Sector")
plt.show()

In [ ]:
# top 10 Employment contributing sectors
industry_wise_summary.sort_values(ascending=False)[:10]

Time Series Employment in Food services and drinking places Sector

In [ ]:
food_sector_data = month_wise_employment_summary[month_wise_employment_summary["LMO_Detailed_Industry"] == "Food services and drinking places"]

food_sector_data.plot(y="Employment", title="Employment in Food services and drinking places Sector evolved overtime", figsize=(20,10))
plt.xlabel("Month and Year")
plt.ylabel("Employment")

Contribution of Employment by Food services and drinking places Sector

In [ ]:
# Calculating the percentage of Employment contributed by Food services and drinking places Sector
food_sector_perc_df = pd.merge(left=total_employment_summary, right=food_sector_data, left_on="month_idx", right_on="month_idx", how="left")
food_sector_perc_df["Employment_perc"] = food_sector_perc_df["Employment_y"] / food_sector_perc_df["Employment_x"] * 100
food_sector_perc_df.head()

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x="month_idx", y="Employment_perc", data=food_sector_perc_df)
plt.xlabel("Year")
plt.ylabel("Employment Percentage")
plt.title("Month wise Employment Percentage Contribution by Food services and drinking places Sector")
plt.show()

Subsector Contribution towards Employment of Food services and drinking places

In [ ]:
lmo_detailed_industries_data[lmo_detailed_industries_data["LMO_Detailed_Industry"] == "Food services and drinking places"]

In [ ]:
# Subsectors contibution towards the Food services and drinking places Sector
# dataframe_4_naics["NAICS"].astype(str).head()
food_subsector_data = dataframe_4_naics[dataframe_4_naics["NAICS"].astype(str).str.match(r'722[0-9]') == True]
food_subsector_summary = food_subsector_data.groupby(["NAICS"])["_EMPLOYMENT_"].sum()
food_subsector_summary = food_subsector_summary.reset_index()
food_subsector_summary.head()

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x="NAICS", y="_EMPLOYMENT_", data=food_subsector_summary)
plt.ylabel("Employment")
plt.title("Employment contribution by Subsector of Food services and drinking places Sector")
plt.show()

Time series Employment Analysis of Repair, personal and non-profit services Sector, the 3rd largest employment contributor

In [ ]:
repair_sector_data = month_wise_employment_summary[month_wise_employment_summary["LMO_Detailed_Industry"] == "Repair, personal and non-profit services"]
# repair_sector_data.head()
repair_sector_data.plot(y="Employment", title="Employment in Repair, personal and non-profit services Sector evolved overtime", figsize=(20,10))
plt.xlabel("Month and Year")
plt.ylabel("Employment")

Contribution of Repair, personal and non-profit services Sector towards total employment

In [ ]:
# Calculating the percentage of Employment contributed by Food services and drinking places Sector
repair_sector_perc_df = pd.merge(left=total_employment_summary, right=repair_sector_data, left_on="month_idx", right_on="month_idx", how="left")
repair_sector_perc_df["Employment_perc"] = repair_sector_perc_df["Employment_y"] / repair_sector_perc_df["Employment_x"] * 100
repair_sector_perc_df.head()

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x="month_idx", y="Employment_perc", data=repair_sector_perc_df)
plt.xlabel("Year")
plt.ylabel("Employment Percentage")
plt.title("Month wise Employment Percentage Contribution by Repair, personal and non-profit services Sector")
plt.show()

Subsector Contribution towards Employment of Repair, personal and non-profit services

In [ ]:
lmo_detailed_industries_data[lmo_detailed_industries_data["LMO_Detailed_Industry"] == "Repair, personal and non-profit services"]

In [ ]:
# Subsectors contibution towards the employment of Repair, personal and non-profit services
repair_subsector_data = dataframe_3_naics[dataframe_3_naics["lower_code"].str.match(r'81[0-9]') == True]
repair_subsector_summary = repair_subsector_data.groupby(["NAICS"])["_EMPLOYMENT_"].sum()
repair_subsector_summary = repair_subsector_summary.reset_index()
repair_subsector_summary.head()

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(x="NAICS", y="_EMPLOYMENT_", data=repair_subsector_summary)
plt.ylabel("Employment")
plt.title("Employment contribution by Subsector of Repair, personal and non-profit services Sector")
plt.show()